The first experiment that I want to do involves the most simple identification of facts within content-free tokens

In [1]:
import torch
import math
from nnsight import CONFIG
from nnsight import LanguageModel
import nnsight
import numpy as np
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv
import random

load_dotenv()

c:\Users\durrc\OneDrive\Desktop\mech_interp_work\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
# importing from my own code 
from activation_transplanting import *

In [3]:
# read the api_key
CONFIG.set_default_api_key(os.environ.get('NDIF_KEY'))

# read the hf token
os.environ['HF_TOKEN'] = os.environ.get('HF_TOKEN')

In [4]:
NDIF_models = [
    "meta-llama/Meta-Llama-3.1-405B-Instruct",
    "meta-llama/Meta-Llama-3.1-8B",
    "meta-llama/Meta-Llama-3.1-70B",
    "deepseek-ai/DeepSeek-R1-Distill-Llama-70B",
    "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
] 

# inexaustive list
non_NDIF_models = [
    "meta-llama/Meta-Llama-3.1-8B",
]

In [5]:
# Example prompts

# instruct examples
prompt_example_1 = "<|begin▁of▁sentence|>\n" \
         "<|start_header_id|>user<|end_header_id|>\n\n" \
         "Hello, how are you? <|eot_id|>\n" \
         "<|start_header_id|>assistant<|end_header_id|>\n"

prompt_example_2 = "<|start_header_id|>system<|end_header_id|>\n\n<|eot_id|>\n" \
                "<|start_header_id|>user<|end_header_id|>\n\n" \
                "Answer the following in one word: What is the tallest mountain in the world?<|eot_id|>\n" \
                "<|start_header_id|>assistant<|end_header_id|>"

# Base model examples 
prompt_example_3 = "\nUser: What's the capital of France?\n\nAssistant:"

# Reasoning examples 
prompt_example_4 = "<｜User｜>Robert has three apples, and then gets one more. How many apples does he have? Respond in a single word.<｜Assistant｜>"

# Basic Replication

Let's do the newline token continuation for base, reasoning, and instruct models 

for each, we'll experiment with a range of prompts to try out. For each, we'll consider variations.

In [7]:
# choose a model 
llama_model_string = "meta-llama/Meta-Llama-3.1-8B"
# remote = use NDIF
remote = True 

if remote and (llama_model_string not in NDIF_models):
    remote = False 
    print("Model not available on NDIF")

# load a model
llama = LanguageModel(llama_model_string)

# commented out for now
tk = LLamaExamineToolkit(
    llama_model=llama, 
    remote=True, # use NDIF
)
"""out = tk.transplant_newline_activities(
    source_strings=["\nUser: Explain fractals in 150 words\n\n",]*5,
    target_strings=['in 150 words\n\n Assistant: ',]*5,
    target_substring='in 150 words\n\n',
    num_new_tokens=100,
    occurrence_index=0,
    num_prev=0,
    num_fut=0,
    transplant_strings=('residual',),
    return_all_tokens=True
    
)"""

'out = tk.transplant_newline_activities(\n    source_strings=["\nUser: Explain fractals in 150 words\n\n",]*5,\n    target_strings=[\'in 150 words\n\n Assistant: \',]*5,\n    target_substring=\'in 150 words\n\n\',\n    num_new_tokens=100,\n    occurrence_index=0,\n    num_prev=0,\n    num_fut=0,\n    transplant_strings=(\'residual\',),\n    return_all_tokens=True\n    \n)'

In [8]:
"""for o in out:
    print(o)"""

'for o in out:\n    print(o)'

# Variations 

We'll now enumerate through different variations to see which one is most likely to produce sentences containing the word "fractal"

In [14]:
# choose a model 
llama_model_string = "meta-llama/Meta-Llama-3.1-8B"
# remote = use NDIF
remote = True 

if remote and (llama_model_string not in NDIF_models):
    remote = False 
    print("Model not available on NDIF")

# load a model
llama = LanguageModel(llama_model_string)

# commented out for now
tk = LLamaExamineToolkit(
    llama_model=llama, 
    remote=True, # use NDIF
)

In [6]:
def generate_transplant_sentences(
        prompt, 
        continuation, 
        newline, 
        tk, 
        samples=5):
    assert newline == continuation[:len(newline)]
    verbosity=tk.verbose
    tk.verbose=False 
    out = tk.transplant_newline_activities(
        source_strings=[prompt,]*samples,
        target_strings=[continuation,]*samples,
        target_substring=newline,
        num_new_tokens=100,
        occurrence_index=0,
        num_prev=0,
        num_fut=0,
        transplant_strings=('residual',),
        return_all_tokens=True
    )
    tk.verbose=verbosity 
    return out


In [16]:
def generate_prompt_structures(topic, add_initial_newline=False, add_newline_after_initial_part=False):
    """
    Generate prompt structures for information storage experiments where newline
    is both the end of the prompt and the beginning of the continuation.
    
    Args:
        topic (str): The topic to explain (e.g., "fractals")
        add_initial_newline (bool): If True, add a \n at the beginning of the prompt
        add_newline_after_initial_part (bool): If True, add a \n after the initial part
        
    Returns:
        list: A list of dictionaries containing prompt structures
    """
    structures = []
    
    # Prepare the common elements
    initial_nl = "\n" if add_initial_newline else ""
    sep = "\n " if add_newline_after_initial_part else " "
    
    # Define template pairs
    templates = [
        ("System: Provide a detailed explanation", " Assistant: "),
        ("Context: Academic explanation requested", " Assistant: "),
        ("Instructions: Be concise and clear", " Response: "),
        ("[Task]", " [Answer]: "),
        ("## Input:", " ## Output:\n"),
        ("# Question", " # Answer\n"),
        ("Human:", " AI: ")
    ]
    
    # Process templates
    for prefix, continuation_suffix in templates:
        # Create the main content part of the prompt
        main_content = f"{initial_nl}{prefix}{sep}Explain {topic} in 150 words"
        
        # Create two options for each template
        # Option 1: newline is just "\n\n"
        newline1 = "\n\n"
        prompt1 = main_content + newline1
        continuation1 = newline1 + continuation_suffix.lstrip()
        
        # Option 2: newline is the full continuation with the newlines
        newline2 = "\n\n" + continuation_suffix.lstrip()
        prompt2 = main_content + newline2
        continuation2 = newline2
        
        # Add both options to structures
        structures.append({
            "prompt": prompt1,
            "continuation": continuation1,
            "newline": newline1
        })
        
        structures.append({
            "prompt": prompt2,
            "continuation": continuation2,
            "newline": newline2
        })
    
    # Handle special case for <prompt> template
    main_content_special = f"{initial_nl}<prompt>Explain {topic} in 150 words</prompt>"
    
    # Option 1: newline is just "\n\n"
    newline_special1 = "\n\n"
    prompt_special1 = main_content_special + newline_special1
    continuation_special1 = newline_special1 + " <response>"
    
    # Option 2: newline is the full continuation
    newline_special2 = "\n\n <response>"
    prompt_special2 = main_content_special + newline_special2
    continuation_special2 = newline_special2
    
    structures.append({
        "prompt": prompt_special1,
        "continuation": continuation_special1,
        "newline": newline_special1
    })
    
    structures.append({
        "prompt": prompt_special2,
        "continuation": continuation_special2,
        "newline": newline_special2
    })
    
    return structures

Let's go through, save all outputs and count the number of intances of the word "fractal"

In [17]:
base_inputs = []

for i in range(2**4):
    base_inputs += generate_prompt_structures(
        topic="fractals",
        add_initial_newline=bool((i//2)%2),
        add_newline_after_initial_part=bool((i//4)%2)
    )

base_inputs[::-1]

[{'prompt': '\n<prompt>Explain fractals in 150 words</prompt>\n\n <response>',
  'continuation': '\n\n <response>',
  'newline': '\n\n <response>'},
 {'prompt': '\n<prompt>Explain fractals in 150 words</prompt>\n\n',
  'continuation': '\n\n <response>',
  'newline': '\n\n'},
 {'prompt': '\nHuman:\n Explain fractals in 150 words\n\nAI: ',
  'continuation': '\n\nAI: ',
  'newline': '\n\nAI: '},
 {'prompt': '\nHuman:\n Explain fractals in 150 words\n\n',
  'continuation': '\n\nAI: ',
  'newline': '\n\n'},
 {'prompt': '\n# Question\n Explain fractals in 150 words\n\n# Answer\n',
  'continuation': '\n\n# Answer\n',
  'newline': '\n\n# Answer\n'},
 {'prompt': '\n# Question\n Explain fractals in 150 words\n\n',
  'continuation': '\n\n# Answer\n',
  'newline': '\n\n'},
 {'prompt': '\n## Input:\n Explain fractals in 150 words\n\n## Output:\n',
  'continuation': '\n\n## Output:\n',
  'newline': '\n\n## Output:\n'},
 {'prompt': '\n## Input:\n Explain fractals in 150 words\n\n',
  'continuation': 

In [18]:
from datetime import datetime 

date_str = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

# Define directory paths
base_dir = f"base_outputs_{date_str}"
fractal_dir = os.path.join(base_dir, "fractal_outputs")

# Create the directories if they don't exist
os.makedirs(fractal_dir, exist_ok=True)



In [19]:
base_inputs[8]

{'prompt': '## Input: Explain fractals in 150 words\n\n',
 'continuation': '\n\n## Output:\n',
 'newline': '\n\n'}

In [20]:
for idx, inputs in enumerate(base_inputs):
    output_sentences = generate_transplant_sentences(tk=tk, **inputs)

    # count_instances of the word fractal 

    num = sum([sent.lower().count("fractal") for sent in output_sentences])

    # File path to save the strings
    file_path = os.path.join(fractal_dir, f"output_count_{num}_index_{idx}.txt")

    # Write the list of strings to the file, each on a new line
    with open(file_path, "w", encoding="utf-8") as file:
        for k, v in inputs.items():
            file.write(f"{k}: {v}" + "\n")
        file.write("--------------- OUTPUTS BELOW ------------" + "\n")
        for string in output_sentences:
            file.write(string + "\n")
        


2025-03-18 12:01:37,698 a7d80888-3314-4d97-b015-4ed9568cf508 - RECEIVED: Your job has been received and is waiting approval.
2025-03-18 12:01:39,200 a7d80888-3314-4d97-b015-4ed9568cf508 - APPROVED: Your job was approved and is waiting to be run.
2025-03-18 12:01:40,264 a7d80888-3314-4d97-b015-4ed9568cf508 - RUNNING: Your job has started running.
2025-03-18 12:01:52,818 a7d80888-3314-4d97-b015-4ed9568cf508 - COMPLETED: Your job has been completed.
2025-03-18 12:02:10,001 f0e4fc0e-d49b-4578-af67-8255104be11d - RECEIVED: Your job has been received and is waiting approval.
2025-03-18 12:02:13,201 f0e4fc0e-d49b-4578-af67-8255104be11d - RUNNING: Your job has started running.
2025-03-18 12:02:13,218 f0e4fc0e-d49b-4578-af67-8255104be11d - APPROVED: Your job was approved and is waiting to be run.
2025-03-18 12:02:18,051 f0e4fc0e-d49b-4578-af67-8255104be11d - COMPLETED: Your job has been completed.
2025-03-18 12:02:23,618 03a407d3-2fa3-41ed-b219-52e69f3cf658 - RECEIVED: Your job has been receive

ChunkedEncodingError: ('Connection broken: IncompleteRead(90210304 bytes read, 18018536 more expected)', IncompleteRead(90210304 bytes read, 18018536 more expected))

In [7]:
# choose a model 
llama_model_string = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
# remote = use NDIF
remote = True 

if remote and (llama_model_string not in NDIF_models):
    remote = False 
    print("Model not available on NDIF")

# load a model
llama = LanguageModel(llama_model_string)

# commented out for now
tk = LLamaExamineToolkit(
    llama_model=llama, 
    remote=True, # use NDIF
)

In [8]:
def generate_reasoning_model_prompts(topic, add_thinking=True, newline1=False, newline2=False, think_only_overlap=False):
    """
    Generate prompt structures for a reasoning model where newline is both:
    1. The end of the prompt
    2. The beginning of the continuation
    
    Args:
        topic (str): The topic to explain (e.g., "fractals")
        add_thinking (bool): If True, add the <think>\n marker after <｜Assistant｜>
        
    Returns:
        list: A list of dictionaries containing prompt structures
    """
    structures = []
    
    # Base user prompt templates
    templates = [
        "Explain {topic} in 150 words",
        "Provide a clear explanation of {topic} in 150 words",
        "What are {topic}? Explain in 150 words",
        "Write a concise explanation of {topic} limited to 150 words",
        "Describe the concept of {topic} in 150 words or less"
    ]
    
    # Instructions that could be prepended
    instructions = [
        "",  # No instruction
        "As an expert, ",
        "Using simple terms, ",
        "In scientific terminology, ",
        "For a beginner audience, "
    ]
    
    # Different possible endings/beginnings (the overlapping "newline" part)
    if add_thinking:
        if think_only_overlap:
            possible_overlaps = [
                "<think>",
                "<think>\n",
                "\n<think>",
                "\n<think>\n"
            ]
            possible_assistants = [
                "<｜Assistant｜>",
                "\n<｜Assistant｜>",
                "<｜Assistant｜>\n",
                "\n<｜Assistant｜>\n",
            ]
        else:
            possible_overlaps = [
                "<｜Assistant｜>",
                "<｜Assistant｜><think>",
                "<｜Assistant｜><think>\n"
                "\n<｜Assistant｜>",
                "\n<｜Assistant｜><think>",
                "\n<｜Assistant｜><think>\n"
                "<｜Assistant｜>\n",
                "<｜Assistant｜>\n<think>",
                "<｜Assistant｜>\n<think>\n"
                "\n<｜Assistant｜>\n",
                "\n<｜Assistant｜>\n<think>",
                "\n<｜Assistant｜>\n<think>\n"
            ]
            possible_assistants=["",]
    else:
        if think_only_overlap:
            assert False, f"think_only_overlap = {think_only_overlap} and add_thinking = {add_thinking}"
        else:
            possible_overlaps = [
                "<｜Assistant｜>"
                "\n<｜Assistant｜>"
                "<｜Assistant｜>\n"
                "\n<｜Assistant｜>\n"
            ]
    
    
    
    # Generate structures
    for template in templates:
        for instruction in instructions:
            for overlap in possible_overlaps:
                for possible_assistant in possible_assistants:
                    # Build the user content
                    user_content = f"{instruction}{template.format(topic=topic)}"

                    n1 = "\n" if newline1 else ""
                    n2 = "\n" if newline2 else ""
                    
                    # Define prompt ending with the overlap
                    prompt = f"{n1}<｜User｜>{n2}{user_content}{possible_assistant}{overlap}"
                    
                    # Define continuation starting with the overlap
                    if add_thinking and overlap == "<｜Assistant｜>":
                        continuation = f"{overlap}<think>\n"
                    elif add_thinking and overlap == "<｜Assistant｜><think>":
                        continuation = f"{overlap}\n"
                    else:
                        continuation = overlap
                    
                    structures.append({
                        "prompt": prompt,
                        "continuation": continuation,
                        "newline": overlap
                    })
                
    return structures

In [9]:
generate_reasoning_model_prompts(topic="fractals", add_thinking=True, newline1=False, newline2=False, think_only_overlap=True)[0]

{'prompt': '<｜User｜>Explain fractals in 150 words<｜Assistant｜><think>',
 'continuation': '<think>',
 'newline': '<think>'}

In [10]:
reasoning_inputs = []

for i in range(2):
    for j in range(2):
        reasoning_inputs += generate_reasoning_model_prompts(
            topic="fractals",
            add_thinking=True,
            newline1=bool(j),
            newline2=bool(i),
            think_only_overlap=True,
        )


reasoning_inputs=reasoning_inputs

In [11]:
from datetime import datetime

date_str = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')


# Define directory paths
reasoning_dir = f"reasoning_outputs_{date_str}"
reasoning_fractal_dir = os.path.join(reasoning_dir, "fractal_outputs")

# Create the directories if they don't exist
os.makedirs(reasoning_fractal_dir, exist_ok=True)

In [12]:
for idx, inputs in enumerate(reasoning_inputs):
    output_sentences = generate_transplant_sentences(tk=tk, **inputs)

    # count_instances of the word fractal 

    num = sum([sent.lower().count("fractal") for sent in output_sentences])

    # File path to save the strings
    file_path = os.path.join(reasoning_fractal_dir, f"output_count_{num}_index_{idx}.txt")

    # Write the list of strings to the file, each on a new line
    with open(file_path, "w", encoding="utf-8") as file:
        for k, v in inputs.items():
            file.write(f"{k}: {v}" + "\n")
        file.write("--------------- OUTPUTS BELOW ------------" + "\n")
        for string in output_sentences:
            file.write(string + "\n")

2025-03-18 22:41:58,553 ddb58b82-857b-4ac9-b898-e91e65a8f122 - RECEIVED: Your job has been received and is waiting approval.
2025-03-18 22:41:59,457 ddb58b82-857b-4ac9-b898-e91e65a8f122 - APPROVED: Your job was approved and is waiting to be run.


: 

In [13]:
for idx, inputs in enumerate(reasoning_inputs):
    output_sentences = generate_transplant_sentences(tk=tk, **inputs)

    # count_instances of the word fractal 

    num = sum([sent.lower().count("fractal") for sent in output_sentences])

    # File path to save the strings
    file_path = os.path.join(reasoning_fractal_dir, f"output_count_{num}_index_{idx}.txt")

    # Write the list of strings to the file, each on a new line
    with open(file_path, "w", encoding="utf-8") as file:
        for k, v in inputs.items():
            file.write(f"{k}: {v}" + "\n")
        file.write("--------------- OUTPUTS BELOW ------------" + "\n")
        for string in output_sentences:
            file.write(string + "\n")

2025-03-18 09:30:05,625 3f359c54-8127-427f-800a-9e9050858960 - RECEIVED: Your job has been received and is waiting approval.
2025-03-18 09:30:06,878 3f359c54-8127-427f-800a-9e9050858960 - APPROVED: Your job was approved and is waiting to be run.
2025-03-18 09:30:07,874 3f359c54-8127-427f-800a-9e9050858960 - RUNNING: Your job has started running.
2025-03-18 09:30:16,173 3f359c54-8127-427f-800a-9e9050858960 - COMPLETED: Your job has been completed.
2025-03-18 09:31:15,361 24d5752e-bd99-432d-9839-5de871b1e5ef - RECEIVED: Your job has been received and is waiting approval.
2025-03-18 09:31:15,771 24d5752e-bd99-432d-9839-5de871b1e5ef - APPROVED: Your job was approved and is waiting to be run.
2025-03-18 09:31:16,388 24d5752e-bd99-432d-9839-5de871b1e5ef - RUNNING: Your job has started running.
2025-03-18 09:31:20,894 24d5752e-bd99-432d-9839-5de871b1e5ef - COMPLETED: Your job has been completed.
2025-03-18 09:31:23,962 fee317d1-5eaa-4547-8e99-00080b8dfc93 - RECEIVED: Your job has been receive

In [36]:
generate_reasoning_model_prompts('fractals', add_thinking=False)

[{'prompt': '<｜User｜>Explain fractals in 150 words<｜Assistant｜>',
  'continuation': '<｜Assistant｜>',
  'newline': '<｜Assistant｜>'},
 {'prompt': '<｜User｜>As an expert, Explain fractals in 150 words<｜Assistant｜>',
  'continuation': '<｜Assistant｜>',
  'newline': '<｜Assistant｜>'},
 {'prompt': '<｜User｜>Using simple terms, Explain fractals in 150 words<｜Assistant｜>',
  'continuation': '<｜Assistant｜>',
  'newline': '<｜Assistant｜>'},
 {'prompt': '<｜User｜>In scientific terminology, Explain fractals in 150 words<｜Assistant｜>',
  'continuation': '<｜Assistant｜>',
  'newline': '<｜Assistant｜>'},
 {'prompt': '<｜User｜>For a beginner audience, Explain fractals in 150 words<｜Assistant｜>',
  'continuation': '<｜Assistant｜>',
  'newline': '<｜Assistant｜>'},
 {'prompt': '<｜User｜>Provide a clear explanation of fractals in 150 words<｜Assistant｜>',
  'continuation': '<｜Assistant｜>',
  'newline': '<｜Assistant｜>'},
 {'prompt': '<｜User｜>As an expert, Provide a clear explanation of fractals in 150 words<｜Assistant｜

In [ ]:
jhgjhg

In [46]:
[llama.tokenizer.decode(w) for w in llama.tokenizer.encode('hello')]

['<|begin_of_text|>', 'hello']

In [47]:
llama.tokenizer

PreTrainedTokenizerFast(name_or_path='meta-llama/Meta-Llama-3.1-8B', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>', 'pad_token': '<|end_of_text|>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128005: AddedToken("

In [9]:
for o in out:
    print(o)

<|begin_of_text|>

 Assistant: 1. What is the sum of the first 1000 prime numbers?

Solution: 1. 2. What is the sum of the first 1000 prime numbers?

Solution: 2. 3. What is the sum of the first 1000 prime numbers?

Solution: 3. 4. What is the sum of the first 1000 prime numbers?

Solution: 4. 5. What is the sum of the first 1000 prime numbers?

Solution
<|begin_of_text|>

 Assistant: 1. What is a 1D array? 2. What is a 2D array? 3. What is a 3D array? 4. What is a 4D array? 5. What is a 5D array? 6. What is a 6D array? 7. What is a 7D array? 8. What is a 8D array? 9. What is a 9D array? 10
<|begin_of_text|>

 Assistant: 1

## What is the answer to the statement "What is the answer to the statement "What is the answer to the statement "What is the answer to the statement "What is the answer to the statement "What is the answer to the statement "What is the answer to the statement "What is the answer to the statement "What is the answer to the statement "What is the answer to the statem

In [10]:
# choose a model 
llama_model_string = "meta-llama/Meta-Llama-3.1-8B"
# remote = use NDIF
remote = True 

if remote and (llama_model_string not in NDIF_models):
    remote = False 
    print("Model not available on NDIF")

# load a model
llama = LanguageModel(llama_model_string)

# commented out for now
tk = LLamaExamineToolkit(
    llama_model=llama, 
    remote=True, # use NDIF
)
out_base = tk.transplant_newline_activities(
    source_strings=["\nUser: Explain fractals in 150 words\n\n",]*5,
    target_strings=['\n\n',]*5,
    target_substring='\n\n',
    num_new_tokens=100,
    occurrence_index=0,
    num_prev=0,
    num_fut=0,
    transplant_strings=('residual','key', 'value', 'output'),
    return_all_tokens=True,
)

extracting token activations


2025-03-16 12:00:31,501 ace6fec6-c524-437d-9e8b-4dc1ce917fc2 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:00:32,257 ace6fec6-c524-437d-9e8b-4dc1ce917fc2 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:00:33,983 ace6fec6-c524-437d-9e8b-4dc1ce917fc2 - RUNNING: Your job has started running.
2025-03-16 12:00:51,182 ace6fec6-c524-437d-9e8b-4dc1ce917fc2 - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

2025-03-16 12:01:30,961 0e68f201-2daf-4197-afc1-067d7c226ee7 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:01:33,497 0e68f201-2daf-4197-afc1-067d7c226ee7 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:01:36,403 0e68f201-2daf-4197-afc1-067d7c226ee7 - RUNNING: Your job has started running.
2025-03-16 12:01:41,789 0e68f201-2daf-4197-afc1-067d7c226ee7 - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

...

2025-03-16 12:02:04,094 4eee0c54-a750-4d1f-b0cc-ac7518e5753a - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:02:05,244 4eee0c54-a750-4d1f-b0cc-ac7518e5753a - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:02:06,909 4eee0c54-a750-4d1f-b0cc-ac7518e5753a - RUNNING: Your job has started running.
2025-03-16 12:02:12,143 4eee0c54-a750-4d1f-b0cc-ac7518e5753a - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

2025-03-16 12:02:18,981 e9416ad0-7927-4506-8d3f-2908ba29b115 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:02:20,514 e9416ad0-7927-4506-8d3f-2908ba29b115 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:02:21,452 e9416ad0-7927-4506-8d3f-2908ba29b115 - RUNNING: Your job has started running.
2025-03-16 12:02:26,020 e9416ad0-7927-4506-8d3f-2908ba29b115 - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

2025-03-16 12:02:31,833 118cf5c5-075f-45e9-bf80-b3ff3604fcf7 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:02:32,725 118cf5c5-075f-45e9-bf80-b3ff3604fcf7 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:02:33,748 118cf5c5-075f-45e9-bf80-b3ff3604fcf7 - RUNNING: Your job has started running.
2025-03-16 12:02:38,401 118cf5c5-075f-45e9-bf80-b3ff3604fcf7 - COMPLETED: Your job has been completed.


In [ ]:
for o in out_base:
    print(o)

NameError: name 'out_base' is not defined

In [ ]:
# choose a model 
llama_model_string = "meta-llama/Meta-Llama-3.1-8B"
# remote = use NDIF
remote = True 

if remote and (llama_model_string not in NDIF_models):
    remote = False 
    print("Model not available on NDIF")

# load a model
llama = LanguageModel(llama_model_string)

# commented out for now
tk = LLamaExamineToolkit(
    llama_model=llama, 
    remote=True, # use NDIF
)
out_base2 = tk.transplant_newline_activities(
    source_strings=["User: Explain fractals in 150 words\n\n",]*5,
    target_strings=['\n\n',]*5,
    target_substring='\n\n',
    num_new_tokens=100,
    occurrence_index=0,
    num_prev=0,
    num_fut=0,
    transplant_strings=('residual','key', 'value', 'output')
)

extracting token activations


2025-03-16 12:03:52,557 6ec92c6b-2875-4010-9063-265d02cba2be - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:03:53,429 6ec92c6b-2875-4010-9063-265d02cba2be - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:03:54,479 6ec92c6b-2875-4010-9063-265d02cba2be - RUNNING: Your job has started running.
2025-03-16 12:04:07,716 6ec92c6b-2875-4010-9063-265d02cba2be - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

2025-03-16 12:04:43,982 aa798639-bc69-4783-aa9c-a7fdbd7313fc - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:04:45,008 aa798639-bc69-4783-aa9c-a7fdbd7313fc - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:04:47,338 aa798639-bc69-4783-aa9c-a7fdbd7313fc - RUNNING: Your job has started running.
2025-03-16 12:04:51,551 aa798639-bc69-4783-aa9c-a7fdbd7313fc - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

...

2025-03-16 12:05:09,185 8df1db42-9e14-47c1-b0d6-e4853f31733c - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:05:09,982 8df1db42-9e14-47c1-b0d6-e4853f31733c - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:05:10,877 8df1db42-9e14-47c1-b0d6-e4853f31733c - RUNNING: Your job has started running.
2025-03-16 12:05:15,447 8df1db42-9e14-47c1-b0d6-e4853f31733c - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

2025-03-16 12:05:21,719 5322fed1-ecf0-4b61-9b42-42c3e94f2632 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:05:23,142 5322fed1-ecf0-4b61-9b42-42c3e94f2632 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:05:24,170 5322fed1-ecf0-4b61-9b42-42c3e94f2632 - RUNNING: Your job has started running.
2025-03-16 12:05:28,404 5322fed1-ecf0-4b61-9b42-42c3e94f2632 - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

2025-03-16 12:05:34,506 68ac1c1a-2cb6-4e13-bf22-981c35eb3d0e - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:05:36,164 68ac1c1a-2cb6-4e13-bf22-981c35eb3d0e - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:05:37,476 68ac1c1a-2cb6-4e13-bf22-981c35eb3d0e - RUNNING: Your job has started running.
2025-03-16 12:05:41,728 68ac1c1a-2cb6-4e13-bf22-981c35eb3d0e - COMPLETED: Your job has been completed.


In [16]:
# choose a model 
llama_model_string = "meta-llama/Meta-Llama-3.1-8B"
# remote = use NDIF
remote = True 

if remote and (llama_model_string not in NDIF_models):
    remote = False 
    print("Model not available on NDIF")

# load a model
llama = LanguageModel(llama_model_string)

# commented out for now
tk = LLamaExamineToolkit(
    llama_model=llama, 
    remote=True, # use NDIF
)
out_base3 = tk.transplant_newline_activities(
    source_strings=["\nUser: Explain fractals in 150 words\n\n",]*5,
    target_strings=['\n\nThe',]*5,
    target_substring='\n\n',
    num_new_tokens=100,
    occurrence_index=0,
    num_prev=0,
    num_fut=0,
    transplant_strings=('residual','key', 'value', 'output')
)

extracting token activations


2025-03-16 12:12:56,659 86512281-a5f8-4a23-8635-e6b4fe289523 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:12:58,082 86512281-a5f8-4a23-8635-e6b4fe289523 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:12:59,725 86512281-a5f8-4a23-8635-e6b4fe289523 - RUNNING: Your job has started running.
2025-03-16 12:13:23,712 86512281-a5f8-4a23-8635-e6b4fe289523 - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

2025-03-16 12:14:05,570 5b8b81bc-244c-41e3-8d01-c6bb68572328 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:14:08,614 5b8b81bc-244c-41e3-8d01-c6bb68572328 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:14:12,205 5b8b81bc-244c-41e3-8d01-c6bb68572328 - RUNNING: Your job has started running.
2025-03-16 12:14:20,211 5b8b81bc-244c-41e3-8d01-c6bb68572328 - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

...

2025-03-16 12:14:44,709 ca692282-5b12-4cd2-8a01-26fd41e81884 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:14:45,787 ca692282-5b12-4cd2-8a01-26fd41e81884 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:14:47,259 ca692282-5b12-4cd2-8a01-26fd41e81884 - RUNNING: Your job has started running.
2025-03-16 12:14:52,046 ca692282-5b12-4cd2-8a01-26fd41e81884 - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

2025-03-16 12:14:58,332 17c056fd-528e-4ac3-926b-d5524299954d - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:14:59,466 17c056fd-528e-4ac3-926b-d5524299954d - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:15:01,813 17c056fd-528e-4ac3-926b-d5524299954d - RUNNING: Your job has started running.
2025-03-16 12:15:08,050 17c056fd-528e-4ac3-926b-d5524299954d - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

2025-03-16 12:15:16,036 d7b73ce4-23f3-425b-bb54-03610d2b114f - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:15:16,854 d7b73ce4-23f3-425b-bb54-03610d2b114f - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:15:18,293 d7b73ce4-23f3-425b-bb54-03610d2b114f - RUNNING: Your job has started running.
2025-03-16 12:15:23,595 d7b73ce4-23f3-425b-bb54-03610d2b114f - COMPLETED: Your job has been completed.


In [17]:
for o in out_base3:
    print(o)

<|begin_of_text|>

The following is a list of all the pages that link to [[:Category:Mathematical physics|Mathematical physics]] and is sorted chronologically (newest first). This list may not reflect recent changes ([[:Category:Mathematical physics|view all]]) ([[:Category:Mathematical physics|edit all]]). 

Jump to: navigation, search

Pages in category "Mathematical physics"

The following 200 pages are in this category, out of 1,019
<|begin_of_text|>

The first part of the course will focus on the theoretical and methodological tools of the history of science. We will read and discuss a selection of texts, among which:

- Peter Dear, The Intelligibility of Nature: How Science Makes Sense of the World (Chicago, 2006)

- Bruno Latour, Science in Action: How to Follow Scientists and Engineers through Society (Cambridge, MA, 1987)

- Steven Shapin and Simon Schaffer, Leviathan and the Air
<|begin_of_text|>

The following is a sample of a student's work for a test in which the student i

In [18]:
# choose a model 
llama_model_string = "meta-llama/Meta-Llama-3.1-8B"
# remote = use NDIF
remote = True 

if remote and (llama_model_string not in NDIF_models):
    remote = False 
    print("Model not available on NDIF")

# load a model
llama = LanguageModel(llama_model_string)

# commented out for now
tk = LLamaExamineToolkit(
    llama_model=llama, 
    remote=True, # use NDIF
)
out_base4 = tk.transplant_newline_activities(
    source_strings=["\nUser: Explain fractals in 150 words\n\n",]*5,
    target_strings=['\n\n',]*5,
    target_substring='\n\n',
    num_new_tokens=100,
    occurrence_index=0,
    num_prev=0,
    num_fut=0,
    transplant_strings=('residual')
)

extracting token activations


2025-03-16 12:19:03,374 1682536f-5d0b-4633-9cf2-a4d13ccf8dce - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:19:05,844 1682536f-5d0b-4633-9cf2-a4d13ccf8dce - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:19:10,394 1682536f-5d0b-4633-9cf2-a4d13ccf8dce - RUNNING: Your job has started running.
2025-03-16 12:19:27,901 1682536f-5d0b-4633-9cf2-a4d13ccf8dce - COMPLETED: Your job has been completed.


generating with transplant
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\

2025-03-16 12:19:40,044 e5f71abe-e31c-4042-907d-198d6b8021a9 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:19:41,591 e5f71abe-e31c-4042-907d-198d6b8021a9 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:19:42,594 e5f71abe-e31c-4042-907d-198d6b8021a9 - RUNNING: Your job has started running.
2025-03-16 12:19:45,463 e5f71abe-e31c-4042-907d-198d6b8021a9 - COMPLETED: Your job has been completed.


generating with transplant
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\

...

2025-03-16 12:20:11,209 23b96c40-ead4-4aa7-9641-5e4af805fe03 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:20:12,690 23b96c40-ead4-4aa7-9641-5e4af805fe03 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:20:13,902 23b96c40-ead4-4aa7-9641-5e4af805fe03 - RUNNING: Your job has started running.
2025-03-16 12:20:18,863 23b96c40-ead4-4aa7-9641-5e4af805fe03 - COMPLETED: Your job has been completed.


generating with transplant
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\

2025-03-16 12:20:22,314 36a38aa7-6e1b-472d-84d2-96c0fbb2b1fa - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:20:24,030 36a38aa7-6e1b-472d-84d2-96c0fbb2b1fa - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:20:24,895 36a38aa7-6e1b-472d-84d2-96c0fbb2b1fa - RUNNING: Your job has started running.
2025-03-16 12:20:28,540 36a38aa7-6e1b-472d-84d2-96c0fbb2b1fa - COMPLETED: Your job has been completed.


generating with transplant
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\

2025-03-16 12:20:33,284 34b8cf7c-04df-4029-a912-1a4259bb8ce5 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:20:34,118 34b8cf7c-04df-4029-a912-1a4259bb8ce5 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:20:35,271 34b8cf7c-04df-4029-a912-1a4259bb8ce5 - RUNNING: Your job has started running.
2025-03-16 12:20:36,396 34b8cf7c-04df-4029-a912-1a4259bb8ce5 - COMPLETED: Your job has been completed.


In [19]:
for o in out_base4:
    print(o)

<|begin_of_text|>

A [[Mathematical Structure]] is a set with some additional structure.

# Examples

* [[Group]]
* [[Ring]]
* [[Vector Space]]
* [[Riemannian Manifold]]
* [[Category]]
* [[Topological Space]]<|end_of_text|>
<|begin_of_text|>

Fractal
A fractal is a natural phenomenon or a mathematical set that exhibits a repeating pattern that displays at every scale. It is also known as expanding symmetry or evolving symmetry. If the replication is exactly the same at every scale, it is called a self-similar pattern. An example of this is the Menger sponge, which is made of densely packed cubes of every size. Fractals are different from other geometric figures because of the way in which they scale. Doubling the edge lengths
<|begin_of_text|>

Fractal: The Golden Ratio

A fractal is a geometric shape that is self-similar and has fractional dimension. The term "fractal" was coined by Benoît Mandelbrot in 1975 and comes from the Latin word fractus meaning "broken" or "fractured." Fracta

In [39]:
tk.produced_plotted_attentions(
        strings=["\nUser: Explain fractals in 150 words\n\n", "User: Explain fractals in 150 words\n\n"]
    )

2025-03-16 12:52:44,036 419e6a63-e558-4325-8704-033ff9a6cabe - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:52:45,007 419e6a63-e558-4325-8704-033ff9a6cabe - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:52:45,748 419e6a63-e558-4325-8704-033ff9a6cabe - RUNNING: Your job has started running.
2025-03-16 12:52:49,388 419e6a63-e558-4325-8704-033ff9a6cabe - COMPLETED: Your job has been completed.


In [ ]:
    source_strings=["\nUser: Explain fractals in 150 words\n\n",]*5,
    target_strings=['\n\n',]*5,
    target_substring='\n\n',
    num_new_tokens=100,
    occurrence_index=0,
    num_prev=0,
    num_fut=0,
    transplant_strings=('residual')

In [40]:
# choose a model 
llama_model_string = "meta-llama/Meta-Llama-3.1-70B"
# remote = use NDIF
remote = True 

if remote and (llama_model_string not in NDIF_models):
    remote = False 
    print("Model not available on NDIF")

# load a model
llama = LanguageModel(llama_model_string)

# commented out for now
tk = LLamaExamineToolkit(
    llama_model=llama, 
    remote=True, # use NDIF
)
out_base = tk.transplant_newline_activities(
    source_strings=["\nUser: Explain fractals in 150 words\n\n",]*5,
    target_strings=['\n\n',]*5,
    target_substring='\n\n',
    num_new_tokens=100,
    occurrence_index=0,
    num_prev=0,
    num_fut=0,
    transplant_strings=('residual','key', 'value', 'output')
)

extracting token activations


2025-03-16 16:30:47,157 e5d3709f-529e-4590-97b0-5dfec226873f - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 16:30:48,143 e5d3709f-529e-4590-97b0-5dfec226873f - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 16:31:06,878 e5d3709f-529e-4590-97b0-5dfec226873f - RUNNING: Your job has started running.
2025-03-16 16:32:08,846 e5d3709f-529e-4590-97b0-5dfec226873f - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

2025-03-16 16:35:20,261 c3b4c8c7-2f50-42fe-b1d3-23a3a77cf357 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 16:35:21,650 c3b4c8c7-2f50-42fe-b1d3-23a3a77cf357 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 16:35:22,567 c3b4c8c7-2f50-42fe-b1d3-23a3a77cf357 - RUNNING: Your job has started running.
2025-03-16 16:35:35,568 c3b4c8c7-2f50-42fe-b1d3-23a3a77cf357 - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

...

2025-03-16 16:36:10,739 56448aea-f03b-4e2d-9ebd-ea4ebf27e109 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 16:36:11,556 56448aea-f03b-4e2d-9ebd-ea4ebf27e109 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 16:36:13,179 56448aea-f03b-4e2d-9ebd-ea4ebf27e109 - RUNNING: Your job has started running.
2025-03-16 16:36:26,011 56448aea-f03b-4e2d-9ebd-ea4ebf27e109 - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

2025-03-16 16:36:38,743 f98237c3-bdb2-4266-9f14-c4ea9cd4494c - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 16:36:39,337 f98237c3-bdb2-4266-9f14-c4ea9cd4494c - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 16:36:40,196 f98237c3-bdb2-4266-9f14-c4ea9cd4494c - RUNNING: Your job has started running.
2025-03-16 16:36:53,191 f98237c3-bdb2-4266-9f14-c4ea9cd4494c - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  11 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

2025-03-16 16:37:01,778 0ad25b6c-e73c-4006-9f0a-a6927386e320 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 16:37:02,229 0ad25b6c-e73c-4006-9f0a-a6927386e320 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 16:37:03,362 0ad25b6c-e73c-4006-9f0a-a6927386e320 - RUNNING: Your job has started running.
2025-03-16 16:37:16,060 0ad25b6c-e73c-4006-9f0a-a6927386e320 - COMPLETED: Your job has been completed.


In [41]:
for o in out_base:
    print(o)

<|begin_of_text|>

Assistant: What can I help you with?

Assistant: What can I help you with?

Assistant: What can I help you with?

Assistant: What can I help you with?

Assistant: What can I help you with?

Assistant: What can I help you with?

Assistant: What can I help you with?

Assistant: What can I help you with?

Assistant: What can I help you with?

Assistant: What can I help you with?

Assistant: What can I help you with?

Assistant
<|begin_of_text|>

AI: The purpose of this blog post is to provide an in-depth analysis of the topic at hand, which is "the importance of effective communication in the workplace". To begin with, it is essential to understand what communication entails. Communication can be defined as the process of exchanging information, ideas, or thoughts between two or more individuals. It involves both verbal and non-verbal cues, such as body language, facial expressions, tone of voice, and gestures.

Effective communication is crucial in the workplace becaus

In [42]:
# choose a model 
llama_model_string = "meta-llama/Meta-Llama-3.1-70B"
# remote = use NDIF
remote = True 

if remote and (llama_model_string not in NDIF_models):
    remote = False 
    print("Model not available on NDIF")

# load a model
llama = LanguageModel(llama_model_string)

# commented out for now
tk = LLamaExamineToolkit(
    llama_model=llama, 
    remote=True, # use NDIF
)
out_base2 = tk.transplant_newline_activities(
    source_strings=["User: Explain fractals in 150 words\n\n",]*5,
    target_strings=['\n\n',]*5,
    target_substring='\n\n',
    num_new_tokens=100,
    occurrence_index=0,
    num_prev=0,
    num_fut=0,
    transplant_strings=('residual','key', 'value', 'output')
)

extracting token activations


2025-03-16 16:37:25,673 1cfe3293-ba0f-4da0-8429-deb0a50fc56d - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 16:37:26,188 1cfe3293-ba0f-4da0-8429-deb0a50fc56d - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 16:37:27,273 1cfe3293-ba0f-4da0-8429-deb0a50fc56d - RUNNING: Your job has started running.
2025-03-16 16:38:17,013 1cfe3293-ba0f-4da0-8429-deb0a50fc56d - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

2025-03-16 16:40:51,023 74422615-700a-494c-97f5-2d2bee38bed5 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 16:40:52,796 74422615-700a-494c-97f5-2d2bee38bed5 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 16:48:17,557 74422615-700a-494c-97f5-2d2bee38bed5 - RUNNING: Your job has started running.
2025-03-16 16:48:20,965 74422615-700a-494c-97f5-2d2bee38bed5 - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

...

2025-03-16 16:48:46,775 ffbf47a9-b8a9-4cc9-b711-0ee276a0c8ab - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 16:48:47,176 ffbf47a9-b8a9-4cc9-b711-0ee276a0c8ab - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 16:48:47,927 ffbf47a9-b8a9-4cc9-b711-0ee276a0c8ab - RUNNING: Your job has started running.
2025-03-16 16:49:00,187 ffbf47a9-b8a9-4cc9-b711-0ee276a0c8ab - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

2025-03-16 16:49:25,080 a7a9397c-4cf2-4768-b675-3b71229a1fcc - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 16:49:25,805 a7a9397c-4cf2-4768-b675-3b71229a1fcc - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 16:49:26,607 a7a9397c-4cf2-4768-b675-3b71229a1fcc - RUNNING: Your job has started running.
2025-03-16 16:49:39,221 a7a9397c-4cf2-4768-b675-3b71229a1fcc - COMPLETED: Your job has been completed.


generating with transplant
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output
residual
source_token =  10 ('\n\n', 271)
target_token =  1 ('\n\n',)
key
value
output

2025-03-16 16:49:48,376 9f2f1ca9-f235-4d0f-a379-7d48a7f8ae4d - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 16:49:48,985 9f2f1ca9-f235-4d0f-a379-7d48a7f8ae4d - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 16:49:49,720 9f2f1ca9-f235-4d0f-a379-7d48a7f8ae4d - RUNNING: Your job has started running.
2025-03-16 16:50:02,280 9f2f1ca9-f235-4d0f-a379-7d48a7f8ae4d - COMPLETED: Your job has been completed.


In [43]:
for o in out_base2:
    print(o)

<|begin_of_text|>

Weegy: It is the study of the physical features of the earth and its atmosphere. (More)<|end_of_text|>
<|begin_of_text|>

Francesca: The word "word" has 4 letters. How many letters are in the word "letter"?

Explanation: The word "letter" has 6 letters.<|end_of_text|>
<|begin_of_text|>

Weegy: The answer is A. The answer is A. The answer is A. The answer is A. The answer is A. The answer is A. The answer is A. The answer is A. The answer is A. The answer is A. The answer is A. The answer is A. The answer is A. The answer is A. The answer is A. The answer is A. The answer is A. The answer is A. The answer is A. The
<|begin_of_text|>

Weegy: It's the ability to make and understand the meaning of sound patterns.

Expert answered|Score.9337| emdjay23 |Points 155736|
User: What is the study of the sounds that make up a language

Weegy: It's the ability to make and understand the meaning of sound patterns.

Expert answered|Score.8871| emdjay23 |Points 155736|
User: What is

# Question:

confirm that residuals is all I need 

the final logit is the same whether I take all tensors, or just residuals 

In [20]:
def extract_final_logits(
        tk,
        source_strings: list[str],
        target_strings: list[str],
        target_substring: str,
        occurrence_index: int = 0,
        num_prev: int = 0,
        num_fut: int = 0,
        transplant_strings: tuple[str] = ("residual"),
    ) -> list[str]:
        """
        extract the logits produced at the final position of target_strings
        """
        assert num_prev >= 0
        assert num_fut >= 0

        # Extract newline activations from source strings
        activation_containers, source_newline_indices = (
            tk.extract_newline_activations(
                strings=source_strings,
                target_substring=target_substring,
                occurrence_index=occurrence_index,
                transplant_strings=transplant_strings,
                num_prev=num_prev,
                num_fut=num_fut,
            )
        )
        
        print("source_newline_indices", source_newline_indices)
        output_logits = []

        # Process each target string with corresponding source activations
        for target_string, activation_container, source_newline_index in zip(
            target_strings, activation_containers, source_newline_indices
        ):
            print("source_newline_index", source_newline_index)
            act = activation_container.get_token_by_index(
                source_newline_index
            )
            print(act)
            
            print(vars(activation_container))

            final_logits = tk.evaluate_with_transplanted_activity(
                target_string=target_string,
                target_substring=target_substring,
                activation_container=activation_container,
                source_token_index=source_newline_index,
                occurrence_index=occurrence_index,
                transplant_strings=transplant_strings,
                num_prev=num_prev,
                num_fut=num_fut,
            )
            output_logits.append(final_logits)
        
        return output_logits

In [36]:
final_logits_1 = extract_final_logits(
        tk,
        source_strings=["\nUser: Explain fractals in 150 words\n\n",],
        target_strings=['\n\n',],
        target_substring='\n\n',
        occurrence_index= -1,
        num_prev = 0,
        num_fut = 0,
        transplant_strings= ("residual",),
    )

extracting token activations


2025-03-16 12:48:56,040 3ae8e2d1-c662-48de-8000-131fa931556a - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:48:57,074 3ae8e2d1-c662-48de-8000-131fa931556a - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:48:58,356 3ae8e2d1-c662-48de-8000-131fa931556a - RUNNING: Your job has started running.
2025-03-16 12:49:00,236 3ae8e2d1-c662-48de-8000-131fa931556a - COMPLETED: Your job has been completed.


source_newline_indices [11]
source_newline_index 11
('\n\n', 271)
{}
generating with transplant
we are transplanting residual
source_token =  11 ('\n\n', 271)
these are toks [128000, 271] 271
target_token =  1 


we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are 

2025-03-16 12:49:04,290 ab7e380b-2d50-4206-b7bd-50681c89b1e7 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:49:04,886 ab7e380b-2d50-4206-b7bd-50681c89b1e7 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:49:06,105 ab7e380b-2d50-4206-b7bd-50681c89b1e7 - RUNNING: Your job has started running.
2025-03-16 12:49:08,934 ab7e380b-2d50-4206-b7bd-50681c89b1e7 - COMPLETED: Your job has been completed.


In [37]:
final_logits_2 = extract_final_logits(
        tk,
        source_strings=["\nUser: Explain fractals in 150 words\n\n",],
        target_strings=['\n\n',],
        target_substring='\n\n',
        occurrence_index= -1,
        num_prev = 0,
        num_fut = 0,
        transplant_strings= ("key", "value"),
    )

extracting token activations


2025-03-16 12:49:13,293 27511de5-1471-425c-8fd1-b262d696b3a8 - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:49:14,541 27511de5-1471-425c-8fd1-b262d696b3a8 - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:49:15,390 27511de5-1471-425c-8fd1-b262d696b3a8 - RUNNING: Your job has started running.
2025-03-16 12:49:16,471 27511de5-1471-425c-8fd1-b262d696b3a8 - COMPLETED: Your job has been completed.


source_newline_indices [11]
source_newline_index 11
('\n\n', 271)
{}
generating with transplant
we are transplanting key
we are transplanting value
source_token =  11 ('\n\n', 271)
these are toks [128000, 271] 271
target_token =  1 


we are transplanting key
we are transplanting value
we are transplanting key
we are transplanting value
we are transplanting key
we are transplanting value
we are transplanting key
we are transplanting value
we are transplanting key
we are transplanting value
we are transplanting key
we are transplanting value
we are transplanting key
we are transplanting value
we are transplanting key
we are transplanting value
we are transplanting key
we are transplanting value
we are transplanting key
we are transplanting value
we are transplanting key
we are transplanting value
we are transplanting key
we are transplanting value
we are transplanting key
we are transplanting value
we are transplanting key
we are transplanting value
we are transplanting key
we are trans

2025-03-16 12:49:19,336 12b14ebe-e7c0-44ee-a8b0-2b9912bed9db - RECEIVED: Your job has been received and is waiting approval.
2025-03-16 12:49:20,111 12b14ebe-e7c0-44ee-a8b0-2b9912bed9db - APPROVED: Your job was approved and is waiting to be run.
2025-03-16 12:49:20,993 12b14ebe-e7c0-44ee-a8b0-2b9912bed9db - RUNNING: Your job has started running.
2025-03-16 12:49:21,948 12b14ebe-e7c0-44ee-a8b0-2b9912bed9db - COMPLETED: Your job has been completed.


In [38]:
torch.isclose(final_logits_1[0], final_logits_2[0]).all()

tensor(False)

[tensor([10.1250, 10.7500, 13.1250,  ..., -4.9375, -4.9375, -4.9375],
        dtype=torch.bfloat16)]

In [ ]:
# choose a model 
llama_model_string = "meta-llama/Meta-Llama-3.1-8B"
# remote = use NDIF
remote = True 

if remote and (llama_model_string not in NDIF_models):
    remote = False 
    print("Model not available on NDIF")

# load a model
llama = LanguageModel(llama_model_string)

# commented out for now
tk = LLamaExamineToolkit(
    llama_model=llama, 
    remote=True, # use NDIF
)
out_base = tk.transplant_newline_activities(
    source_strings=["\nUser: Explain fractals in 150 words\n\n",]*5,
    target_strings=['\n\n',]*5,
    target_substring='\n\n',
    num_new_tokens=100,
    occurrence_index=0,
    num_prev=0,
    num_fut=0,
    transplant_strings=('residual','key', 'value', 'output')
)

In [ ]:
jhg

NameError: name 'jhg' is not defined

In [10]:
for o in out:
    print(o)

<|begin_of_text|>

 Assistant: 1

 Solution

Solution 1

Let the number be $x$. Then, $x$ is divisible by $3$ and $4$ and also by $12$.

Therefore, $x$ is divisible by $3\times4\times12=144$.

Hence, the number is $144$.

Solution 2

Let the number be $x$. Then, $x$ is divisible by $3$ and $4$ and also by $12$.


<|begin_of_text|>

 Assistant: 1. Write the equation of a line that passes through the point (1, 2) and is parallel to the line y = 3x + 5. 2. Write the equation of a line that passes through the point (1, 2) and is perpendicular to the line y = 3x + 5. 3. Write the equation of a line that passes through the point (1, 2) and is parallel to the line y = 3x
<|begin_of_text|>

 Assistant:  What is the sum of the first 50 positive integers?<|end_of_text|>
<|begin_of_text|>

 Assistant:  In the previous lesson, you learned about the different types of variables that can be used in a Python program. In this lesson, you will learn about the different types of functions that can be us

In [ ]:
jhg

In [ ]:
# choose a model 
llama_model_string = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B" # "meta-llama/Meta-Llama-3.1-70B"
# remote = use NDIF
remote = True 

if remote and (llama_model_string not in NDIF_models):
    remote = False 
    print("Model not available on NDIF")

# load a model
llama = LanguageModel(llama_model_string)

# commented out for now
tk = LLamaExamineToolkit(
    llama_model=llama, 
    remote=True, # use NDIF
)
run_simple_number_experiment(tk, num=10, number_samples=1,target_substring="<｜Assistant｜>", mode='reason')

Source:  <｜User｜>Word Problem Setup: A girl has ten plums when she leaves the grocery store<｜Assistant｜> 
Question: <｜Assistant｜> Therefore, the total number of plums purchased was 
extracting token activations


2025-03-15 16:50:35,516 bd95cdcf-1419-48a2-8032-fc74e9adb024 - RECEIVED: Your job has been received and is waiting approval.
2025-03-15 16:50:36,025 bd95cdcf-1419-48a2-8032-fc74e9adb024 - APPROVED: Your job was approved and is waiting to be run.
2025-03-15 16:50:36,900 bd95cdcf-1419-48a2-8032-fc74e9adb024 - RUNNING: Your job has started running.
2025-03-15 16:50:37,764 bd95cdcf-1419-48a2-8032-fc74e9adb024 - COMPLETED: Your job has been completed.


source_newline_indices [18]
source_newline_index 18
('<｜Assistant｜>', 128012)
{}
generating with transplant
we are transplanting residual
source_token =  18 ('<｜Assistant｜>', 128012)
these are toks [128000, 128012, 15636, 11, 279, 2860, 1396, 315, 628, 6370, 15075, 574, 220] 128012
target_token =  1 <｜Assistant｜>
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting residual
we are transplanting resi

2025-03-15 16:50:40,453 9cac7e92-fea0-4c1d-a714-171ef30d856e - RECEIVED: Your job has been received and is waiting approval.
2025-03-15 16:50:41,149 9cac7e92-fea0-4c1d-a714-171ef30d856e - APPROVED: Your job was approved and is waiting to be run.
2025-03-15 16:50:41,567 9cac7e92-fea0-4c1d-a714-171ef30d856e - RUNNING: Your job has started running.
2025-03-15 16:50:42,243 9cac7e92-fea0-4c1d-a714-171ef30d856e - COMPLETED: Your job has been completed.


array([0.11493165, 0.5836711 , 0.2147206 , 0.03616485, 0.02193509,
       0.01372663, 0.00431928, 0.00347064, 0.00133805, 0.00572211],
      dtype=float32)

In [9]:
with llama.generate(
        '<｜User｜>Word Problem Setup: A woman has ten oranges when he leaves the market.<｜Assistant｜> The total number of oranges was ',
        max_new_tokens=20,
        remote=True,
    ) as tracer:
    out = llama.generator.output.save()


2025-03-15 16:50:43,847 73733c63-b7d8-488d-a57f-19277fa8daaa - RECEIVED: Your job has been received and is waiting approval.
2025-03-15 16:50:44,526 73733c63-b7d8-488d-a57f-19277fa8daaa - APPROVED: Your job was approved and is waiting to be run.
2025-03-15 16:50:45,235 73733c63-b7d8-488d-a57f-19277fa8daaa - RUNNING: Your job has started running.
2025-03-15 16:50:46,419 73733c63-b7d8-488d-a57f-19277fa8daaa - COMPLETED: Your job has been completed.


In [10]:
tk.transplant_newline_activities(
    source_strings=["<｜User｜>Word Problem: A man has one apple. Then he loses one. Then he finds one more apple. If he gains two more apples, how many will he have?<｜Assistant｜>",]*5,
    target_strings=['<｜Assistant｜>',]*5,
    target_substring='<｜Assistant｜>',
    num_new_tokens=100,
    occurrence_index=0,
    num_prev=0,
    num_fut=0,
    transplant_strings=('residual',)
)


extracting token activations


2025-03-15 16:50:48,693 c666cfcf-580f-442c-9d66-5963923bcccc - RECEIVED: Your job has been received and is waiting approval.
2025-03-15 16:50:49,127 c666cfcf-580f-442c-9d66-5963923bcccc - APPROVED: Your job was approved and is waiting to be run.
2025-03-15 16:50:49,513 c666cfcf-580f-442c-9d66-5963923bcccc - RUNNING: Your job has started running.
2025-03-15 16:51:04,361 c666cfcf-580f-442c-9d66-5963923bcccc - COMPLETED: Your job has been completed.


generating with transplant
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_to

2025-03-15 16:51:39,926 92397b39-c8ef-4c1a-8095-c43657fdc2df - RECEIVED: Your job has been received and is waiting approval.
2025-03-15 16:51:40,815 92397b39-c8ef-4c1a-8095-c43657fdc2df - APPROVED: Your job was approved and is waiting to be run.
2025-03-15 16:51:41,788 92397b39-c8ef-4c1a-8095-c43657fdc2df - RUNNING: Your job has started running.
2025-03-15 16:51:44,607 92397b39-c8ef-4c1a-8095-c43657fdc2df - COMPLETED: Your job has been completed.


generating with transplant
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_to

...

generating with transplant
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_to

2025-03-15 16:52:06,280 cfed7404-8d84-40bc-a065-bd361d5d0ef6 - RECEIVED: Your job has been received and is waiting approval.
2025-03-15 16:52:07,190 cfed7404-8d84-40bc-a065-bd361d5d0ef6 - APPROVED: Your job was approved and is waiting to be run.
2025-03-15 16:52:08,144 cfed7404-8d84-40bc-a065-bd361d5d0ef6 - RUNNING: Your job has started running.
2025-03-15 16:52:11,144 cfed7404-8d84-40bc-a065-bd361d5d0ef6 - COMPLETED: Your job has been completed.


generating with transplant
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_token =  1 ('<｜Assistant｜>',)
residual
source_token =  36 ('<｜Assistant｜>', 128012)
target_to

2025-03-15 16:52:14,878 d472e331-343c-4917-997d-a6bf2fbe22dd - RECEIVED: Your job has been received and is waiting approval.
2025-03-15 16:52:16,233 d472e331-343c-4917-997d-a6bf2fbe22dd - APPROVED: Your job was approved and is waiting to be run.
2025-03-15 16:52:17,715 d472e331-343c-4917-997d-a6bf2fbe22dd - RUNNING: Your job has started running.
2025-03-15 16:52:21,624 d472e331-343c-4917-997d-a6bf2fbe22dd - COMPLETED: Your job has been completed.


["<｜begin▁of▁sentence｜><｜Assistant｜><think>\n\n</think>\n\nSure! Could you clarify or provide more details about what you'd like assistance with? Whether it's a question, a problem to solve, or something else, feel free to share, and I'll do my best to help. 😊<｜end▁of▁sentence｜>",
 '<｜begin▁of▁sentence｜><｜Assistant｜><think>\n\n</think>\n\n**Question:**  \nWhat is the answer to the question: "What is the square root of 64?" \n\n**Answer:**  \nThe square root of 64 is **8**.<｜end▁of▁sentence｜>',
 '<｜begin▁of▁sentence｜><｜Assistant｜><think>\nAlright, so I\'ve got this riddle here: "What has keys but can\'t open locks?" Hmm, at first glance, it seems a bit tricky, but maybe if I break it down, I can figure it out. Let me think about what I know about keys and locks. Keys are typically used to open locks, right? So if something has keys, you might think it\'s related to opening things. But the question says it can\'t open locks, which is confusing',
 "<｜begin▁of▁sentence｜><｜Assistant｜><think

In [11]:

llama.tokenizer.decode(out[0])

"<｜begin▁of▁sentence｜><｜User｜>Word Problem Setup: A woman has ten oranges when he leaves the market.<｜Assistant｜> The total number of oranges was 10.\n\nOkay, so I've got this problem where a woman has ten oranges when she leaves the"

In [ ]:
for o in out:
    print()

In [ ]:
jhg

NameError: name 'jhg' is not defined

In [9]:
# choose a model 
llama_model_string = "meta-llama/Meta-Llama-3.1-405B-Instruct"
# remote = use NDIF
remote = True 

if remote and (llama_model_string not in NDIF_models):
    remote = False 
    print("Model not available on NDIF")

# load a model
llama = LanguageModel(llama_model_string)

# commented out for now
tk = LLamaExamineToolkit(
    llama_model=llama, 
    remote=True, # use NDIF
)

run_simple_number_experiment(tk, num=10, number_samples=1,target_substring='<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n', mode='instruct')

<|start_header_id|>user<|end_header_id|>

Word Problem Setup: A boy has ten pears when he leaves the grocery store<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
 <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
 Therefore, the total number of pears they currently had was
extracting token activations


2025-03-15 15:32:07,178 fb113c8b-545f-440f-9506-305235816b37 - RECEIVED: Your job has been received and is waiting approval.
2025-03-15 15:32:08,841 fb113c8b-545f-440f-9506-305235816b37 - APPROVED: Your job was approved and is waiting to be run.


: 

: 

In [25]:
generate_random_simple_number_string(10)

'Word Problem Setup: A woman has ten oranges when he leaves the market.\n\n'

In [ ]:
with llama.

In [9]:
with llama.generate(
            'Word Problem Setup: A woman has ten oranges when he leaves the market.\n\n The total number of oranges was ',
            max_new_tokens=20,
            remote=True,
        ) as tracer:
        out = llama.generator.output.save()
    

ConnectionError: Internal Server Error

: 

In [ ]:
prompt = 'The Eiffel Tower is in the city of'
n_new_tokens = 3
with llama.generate(prompt, max_new_tokens=n_new_tokens, remote=True) as tracer:
    out = llama.generator.output.save()

2025-03-14 17:04:25,407 003a8792-56b6-4057-9a44-6d46f24a10b7 - RECEIVED: Your job has been received and is waiting approval.
2025-03-14 17:04:25,927 003a8792-56b6-4057-9a44-6d46f24a10b7 - APPROVED: Your job was approved and is waiting to be run.
2025-03-14 17:04:27,093 003a8792-56b6-4057-9a44-6d46f24a10b7 - RUNNING: Your job has started running.
2025-03-14 17:04:27,652 003a8792-56b6-4057-9a44-6d46f24a10b7 - COMPLETED: Your job has been completed.


In [44]:
llama.tokenizer.decode(out[0])

'<|begin_of_text|>Word Problem Setup: A woman has ten oranges when he leaves the market.\n\n The total number of oranges was 10. A woman has ten oranges when he leaves the market. He gives three to his daughter.'

In [27]:
outputs=tk.transplant_newline_activities(
        source_strings=['Word Problem Setup: A woman has ten oranges when he leaves the market.\n\n She has '],
        target_strings=['Word Problem Setup: A woman has ten oranges when he leaves the market.\n\n She has '],
        num_new_tokens=10,
        target_substring=".\n\n",
        occurrence_index= 0,
        num_prev = 0,
        num_fut= 0,
        transplant_strings= ("residual",),
    )


extracting token activations


2025-03-14 17:24:11,486 7a1970e4-1246-42eb-a17f-bbc072cfc38b - RECEIVED: Your job has been received and is waiting approval.
2025-03-14 17:24:11,930 7a1970e4-1246-42eb-a17f-bbc072cfc38b - APPROVED: Your job was approved and is waiting to be run.
2025-03-14 17:24:12,520 7a1970e4-1246-42eb-a17f-bbc072cfc38b - RUNNING: Your job has started running.
2025-03-14 17:24:14,806 7a1970e4-1246-42eb-a17f-bbc072cfc38b - COMPLETED: Your job has been completed.


generating with transplant


AttributeError: 'tuple' object has no attribute 'replace'

In [24]:
run_simple_number_experiment(tk, num=10, number_samples=1)

extracting token activations


2025-03-14 16:54:10,082 eaa305d2-3b3e-49cc-8288-593753fe095c - RECEIVED: Your job has been received and is waiting approval.
2025-03-14 16:54:11,607 eaa305d2-3b3e-49cc-8288-593753fe095c - APPROVED: Your job was approved and is waiting to be run.
2025-03-14 16:54:13,479 eaa305d2-3b3e-49cc-8288-593753fe095c - RUNNING: Your job has started running.
2025-03-14 16:54:19,099 eaa305d2-3b3e-49cc-8288-593753fe095c - COMPLETED: Your job has been completed.


source_newline_indices [16]
source_newline_index 16
('.\n\n', 382)
{}
generating with transplant
source_token =  11 (' when', 994)
these are toks [128000, 11116, 22854, 19139, 25, 362, 893, 706, 5899, 1069, 14576, 994, 568, 11141, 279, 3637, 382, 382, 7009, 617] 11116
target_token =  -4 .


source_token =  12 (' he', 568)
these are toks [128000, 11116, 22854, 19139, 25, 362, 893, 706, 5899, 1069, 14576, 994, 568, 11141, 279, 3637, 382, 382, 7009, 617] 11116
target_token =  -3 .


source_token =  13 (' leaves', 11141)
these are toks [128000, 11116, 22854, 19139, 25, 362, 893, 706, 5899, 1069, 14576, 994, 568, 11141, 279, 3637, 382, 382, 7009, 617] 11116
target_token =  -2 They
source_token =  14 (' the', 279)
these are toks [128000, 11116, 22854, 19139, 25, 362, 893, 706, 5899, 1069, 14576, 994, 568, 11141, 279, 3637, 382, 382, 7009, 617] 11116
target_token =  -1  have
source_token =  15 (' store', 3637)
these are toks [128000, 11116, 22854, 19139, 25, 362, 893, 706, 5899, 1069, 14576, 

2025-03-14 16:54:38,784 4f43751e-117a-4c53-ad48-b664cd88ab7b - RECEIVED: Your job has been received and is waiting approval.
2025-03-14 16:54:43,241 4f43751e-117a-4c53-ad48-b664cd88ab7b - APPROVED: Your job was approved and is waiting to be run.
2025-03-14 16:54:49,764 4f43751e-117a-4c53-ad48-b664cd88ab7b - RUNNING: Your job has started running.
2025-03-14 16:54:57,709 4f43751e-117a-4c53-ad48-b664cd88ab7b - COMPLETED: Your job has been completed.


array([0.11248883, 0.1583274 , 0.04535482, 0.08479982, 0.13980855,
       0.03025266, 0.04424512, 0.04272186, 0.01677323, 0.32522765],
      dtype=float32)

First, we'll see if we can read from this how many fruits there were 

Now lets generate a bunch of random strings for each number in 1-10 

we'll try to see if we can extract from this the placement of the vectors 